In [1]:
#Klassifikation Meht3 LGBM-Classifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import pandas as pd


In [2]:
df_accidents = pd.read_csv('../../data/MA3_finished.csv', index_col='ID')

In [3]:
df_accidents.dtypes

Unnamed: 0                 int64
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Temperature(F)           float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Weather_Condition         object
Amenity                     bool
Bump                        bool
Crossing                    bool
Give_Way                    bool
Junction                    bool
No_Exit                     bool
Railway                     bool
Roundabout

In [4]:
#featureselection
#features to encode
obj_bool_features = df_accidents[['Division','Region','State Name','Astronomical_Twilight','Nautical_Twilight','Civil_Twilight',
                               'Sunrise_Sunset','Weather_Condition','Sunrise_Sunset','Airport_Code','Timezone','Country','Zipcode',
                               'State','County', 'City' ,'Street','Description','Start_Time', 'End_Time', 'Weather_Condition', 
                               'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 
                               'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset']]

In [5]:
#encode features above
le = LabelEncoder()
#create copy of df
df_encoded = pd.DataFrame()
#LabelEncoder can only encode one column at a time --> forloop
#obj_bool_feat = le.fit_transform(obj_bool_features)
for feature in obj_bool_features:
    df_encoded[feature] = le.fit_transform(df_accidents[feature])

In [6]:
#float/int features
float_int_features = df_accidents[['Severity','Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',]]

for feature in float_int_features:
    df_encoded[feature] = df_accidents[feature]

#check if combination worked 
df_encoded.columns

Index(['Division', 'Region', 'State Name', 'Astronomical_Twilight',
       'Nautical_Twilight', 'Civil_Twilight', 'Sunrise_Sunset',
       'Weather_Condition', 'Airport_Code', 'Timezone', 'Country', 'Zipcode',
       'State', 'County', 'City', 'Street', 'Description', 'Start_Time',
       'End_Time', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction',
       'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Severity',
       'Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)'],
      dtype='object')

In [7]:
#check if encoding worked 
print(df_encoded)

         Division  Region  State Name  Astronomical_Twilight  \
0               0       0          33                      0   
1               0       0          33                      0   
2               0       0          33                      0   
3               0       0          33                      0   
4               0       0          33                      0   
...           ...     ...         ...                    ...   
7051551         5       3           3                      0   
7051552         5       3           3                      0   
7051553         5       3           3                      0   
7051554         5       3           3                      0   
7051555         5       3           3                      0   

         Nautical_Twilight  Civil_Twilight  Sunrise_Sunset  Weather_Condition  \
0                        0               1               1                 86   
1                        0               0               1           

In [8]:
#check amount of severity for each level
print(df_encoded['Severity'].value_counts())

Severity
2    5671502
3    1136465
4     178538
1      65051
Name: count, dtype: int64


In [9]:
#set Target
target = df_encoded['Severity']
#set features
features = df_encoded.drop(columns=['Severity'])
#scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [10]:
#Split Data 80/20
features_train, features_test, target_train, target_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

In [11]:
#lgbm-classifier
lgbm_classifier = LGBMClassifier(n_estimator = 50, random_state=42)

#train classifier
lgbm_classifier.fit(features_train, target_train)

#predict
target_predict = lgbm_classifier.predict(features_test)

C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.296925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3432
[LightGBM] [Info] Number of data points in the train set: 5641244, number of used features: 37
[LightGBM] [Info] Start training from score -4.687059
[LightGBM] [Info] Start training from score -0.217710
[LightGBM] [Info] Start training from score -1.825832
[LightGBM] [Info] Start training from score -3.675211
[LightGBM] [Warning] Unknown parameter: n_estimator


In [12]:
accuracy = accuracy_score(target_test, target_predict)
print('Accuracy= ', {accuracy})
print(classification_report(target_test, target_predict))
print(confusion_matrix(target_test, target_predict))

Accuracy=  {0.9271515806431485}
              precision    recall  f1-score   support

           1       0.77      0.64      0.70     13074
           2       0.94      0.97      0.96   1133919
           3       0.86      0.78      0.82    227753
           4       0.94      0.47      0.63     35566

    accuracy                           0.93   1410312
   macro avg       0.88      0.72      0.77   1410312
weighted avg       0.93      0.93      0.92   1410312

[[   8337    4445     292       0]
 [   2136 1105571   25619     593]
 [    321   50012  176867     553]
 [     37   16247    2484   16798]]


In [13]:
#Teilen Sie dazu zunächst die Daten auf, um Overfitting beim Trainieren des Algorithmus und bei der Parameterauswahl zu vermeiden. Erklären Sie die gewählte Strategie und die Größenverhältnisse.
#Um Overfitting zu vermeiden haben wir die Train-Test-Split Strategie gewählt. Diese teilt den Datensatz in
#Trainingsdaten und Testdaten zufällig auf. In verschiedenen Recherchen konnten wir feststellen, dass die Größenverhältnisse
#80/20 oft verwendet werden. Wir sehen dies nicht zwingend als Standard, haben uns aber entschieden hier ähnlich vorzugehen.
#Da die Accuracy einen guten Wert hat und somit unser Model einen großteil der Instanzen richtig klassifiziert, denken wir dass unserer aufteilung ausreichend gut ist.

#Wählen Sie geeignete Features aus und setzen Sie die Parameter des Algorithmus. Beschreiben Sie das gewälhte Vorgehen für die Auswahl der Features und Parameter. Berichten Sie den Parameterraum und die final gewählten Parameter. Geben Sie die Performanz auf den Trainingsdaten (bzw. Entwicklungsdaten, falls verwendet) an.
#Die Features wurden in iterativ durch ausprobieren gewählt. Es wurde verscuht möglichst viele features mit einzubeziehen.
#Es kam zu Porblemen bei den Spalten ZipCode und AirportCode. Diese Spalten wurden rausgenommen um speicherprobleme(RAM) zu vermeiden.

#In weiteren Tests ist uns hierzu aufgefallen,dass dieser Classifier fast alle features aufnehemen und mit einbeziehen kann.

#der Classifier der hier verwendet wird braucht nur wenige Parameter. Übergeben wurde die Anzahl der Estimator (Wir sind uns hier nicht sicher, ob dieser Wert genommen wurde --> siehe Warning)
#random_state is der sogennate Seed. 42 ist hier ein Standardwert. Dadurch ist der Code reproduzierbar. 
# Die Performanz auf den Daten ist ersichtlicht in der letzten Ausgabe

#Evaluieren Sie die Klassifikation auf den ungesehenen Testdaten. Betrachten Sie Precision und Recall sowie den F-Wert. Welches Maß ist für Ihre Anwendung wichtiger? Bewerten Sie Ihr Ergebnis. Ist es in der Praxis voraussichtlich zufriedenstellend?
#In unserem Fall möchten wir sicherstellen, dass schwere Unfälle sicher zur Kategorie schwer gehören (Verhältnis truepositives und allpositives), 
#Daher ist in unserem Fall die Precission wichtiger. 
